# Extracting insights from courses & majors

same deal with professors, know what these descriptions and what not intends to tell you. What are the types of information that is contained in these scraped datas? 

- perceived prerequisites: what is actually needed to understand these. NOT the hard-written prerequisites list because that can be misleading and inaccurate, since some concepts are rarely used.
- contents: what is taught in this course? 
- outcomes: What can you now achieve from taking this class that you previously cannot? How does this class benefit you in terms of academic & career & personal growth goals? How does it fit into the prerequisites chain? 

with these categories, we can prompt engineer our LLM to extract that.

along with that job, i also want to batch in another task while we're at it:
- hard written prerequisites: output as a list of course codes string is fine
- distribution requirements: a list (multiselect) from a predefined set of distributions, rest is ignored.
- combined with: output a list of course codes from the natural language input.

In [ ]:
EXTRACTION_COURSE_DESCRIPTION_PROMPT = """
### HARD_WRITTEN_PREREQUISITES
{prereqs}

### DESCRIPTION
{description}
#### said outcomes
{outcomes}

### OTHER INPUT INFORMATIONS
combined_with: {combined_with}
distribution_requirements: {distributions}

### INSTRUCTION
- perceived_prerequisites: what is actually needed to understand these. NOT the hard-written prerequisites list because that can be misleading and inaccurate, since some concepts are rarely used. output a list of short and concise string capturing everything. short sentences suffice
- contents: what is taught in this course? output a list of short and concise string capturing everything. short sentences suffice
- outcomes: What can you now achieve from taking this class that you previously cannot? How does this class benefit you in terms of academic & career & personal growth goals? How does it fit into the prerequisites chain? output a list of short and concise string capturing everything. short sentences suffice
- hard_written_prerequisites: output as a list of course codes string is fine
- distributions: a list (multiselect) from a predefined set of distributions, rest is ignored.
- combined_with: output a list of course codes from the natural language input.

### OUTPUT REQUIREMENTS
DO NOT MAKE UP ANY INFORMATION.
If there is not enough information to extract these information, output "[]" as an empty JSON list.
JSON output must be compatible and must begin with {{ and end with }} so that python's json.loads can parse it.
No escape character is needed for ', and wrap string with double quotes.

### JSON DATA EXTRACTED (beginning with {{ and ending with }}):
"""

In [ ]:
title = "Object-Oriented Programming and Data Structures"